In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
data = np.genfromtxt("build/time.csv", skip_header=1, delimiter=',')
data[:, 2] -= data[0, 1]
data[:, 1] -= data[0, 1]

In [3]:
def normalize(x):
    return x / np.amax(np.abs(x))

In [1]:
world_size = int(data[-1,0])+1

plt.figure(figsize=(9,8/5*world_size))


for proc in range(0, world_size):
    X = data[data[:, 0] == proc, :]
    if proc == 0:
        ax0 = plt.subplot(world_size, 1, proc+1)
    else:
        ax = plt.subplot(world_size, 1, proc+1, sharex=ax0)
    
    t_tot = np.sum(X[:, 3:], axis=1)/100.0
    plt.stackplot(X[:, 1]/60, X[:,3]/t_tot, X[:,4]/t_tot, X[:,5]/t_tot, X[:,6]/t_tot, labels=["Comp", "Send", "Recv", "Idle"])
    
    
    if proc == world_size-1:
        plt.legend()
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        #ax.spines['left'].set_visible(False)
        plt.xlabel("Minutes")
        plt.ylabel("Percent")
    else:
        plt.axis("off")

plt.show()

NameError: name 'data' is not defined

In [5]:
np.amax(X[:, 2])

12.248466968536377

In [6]:
(np.amax(X[:,1])-np.amin(X[:, 1]))/3600

0.003401955034997728